In [1]:
import climlab
import numpy as np
import matplotlib.pyplot as plt

In [2]:
model_dims = {'lat':20, 'lon':10, 'lev':30}

In [ ]:
def make_volume_domains(num_lev=30, num_lat=30, num_lon=30, water_depth=1.0, **kwargs):
    '''
    Create 3D volumes
    '''
    
    # make axes
    latax = climlab.domain.axis.Axis(axis_type='lat', num_points=num_lat)
    lonax = climlab.domain.axis.Axis(axis_type='lon', num_points=num_lon)
    levax = climlab.domain.axis.Axis(axis_type='lev', num_points=num_lev)
    depax = climlab.domain.axis.Axis(axis_type='depth', bounds=[water_depth, 0.0])
    
    # make ocean
    slab = climlab.domain.domain.SlabOcean(axes={'lat':latax, 'lon':lonax, 'depth':depax}, **kwargs)
    
    # make atmosphere
    atm = climlab.domain.domain.Atmosphere(axes={'lat':latax, 'lon':lonax, 'lev':levax}, **kwargs)
    
    # return the domains
    return slab, atm
    
def volume_state(num_lev=30, num_lat=30, num_lon=30, water_depth=1.0):
    '''
    Create a 3D climlab state
    '''
    
    # make domains
    sfc, atm = make_volume_domains(num_lev=num_lev, num_lat=num_lat, num_lon=num_lon, water_depth=water_depth)
    
    # update elevation value
    num_lev = atm.lev.num_points
    
    # make fields
    Ts = climlab.domain.field.Field(288.*np.ones(sfc.shape), domain=sfc)
    Tinitial = np.tile(np.linspace(200., 288.-10., num_lev), sfc.shape)
    Tatm = climlab.domain.field.Field(Tinitial, domain=atm)
    
    # create state
    state = climlab.utils.attrdict.AttrDict()
    state['Ts'] = Ts
    state['Tatm'] = Tatm
    
    # return the state
    return state    

In [4]:
# make state
state = volume_state(num_lat=model_dims['lat'], num_lon=model_dims['lon'], num_lev=model_dims['lev'])

In [3]:
model = climlab.radiation.DailyInsolation(name='Insolation', domains=state['Ts'].domain) 

In [51]:
model.state

AttrDict({})

In [ ]:
state['Tatm'].domain.axis_index['lev']

In [ ]:
state['Tatm'].shape

In [ ]:
model.compute_diagnostics()

In [ ]:
model.insolation.shape

In [ ]:
plt.plot(model.lon, np.squeeze(model.insolation)[5,:], label='Surface')
plt.plot(model.lon, model.insolation[5,:,-1], label='Atmosphere')
plt.gca().ticklabel_format(useOffset=False)
plt.xlabel('Longitude')
plt.ylabel('Temperature (K)')
plt.legend()
plt.show()

In [ ]:
plt.plot(model.lat, np.squeeze(model.insolation)[:,5], label='Surface')
plt.plot(model.lat, model.insolation[:,5,-1], label='Atmosphere')
plt.gca().ticklabel_format(useOffset=False)
plt.xlabel('Latitude')
plt.ylabel('Temperature (K)')
plt.legend()
plt.show()

In [ ]:
state = volume_state(num_lat=model_dims['lat'], num_lon=model_dims['lon'], num_lev=model_dims['lev'])

In [ ]:
state

In [5]:
ebm = climlab.EBM(state=state)

/home/henryjmo/Documents/_Feldl/climlab/climlab/domain/field.py:278: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if axis == 'lat':


ValueError: operands could not be broadcast together with shapes (10,30,21) (30,) 

In [6]:
model_dims

{'lat': 20, 'lon': 10, 'lev': 30}

In [24]:
a = np.linspace(0,9,30)

In [25]:
a

array([0.        , 0.31034483, 0.62068966, 0.93103448, 1.24137931,
       1.55172414, 1.86206897, 2.17241379, 2.48275862, 2.79310345,
       3.10344828, 3.4137931 , 3.72413793, 4.03448276, 4.34482759,
       4.65517241, 4.96551724, 5.27586207, 5.5862069 , 5.89655172,
       6.20689655, 6.51724138, 6.82758621, 7.13793103, 7.44827586,
       7.75862069, 8.06896552, 8.37931034, 8.68965517, 9.        ])

In [11]:
b = np.tile(a, (5,1),)

In [14]:
b.shape

(5, 10)

In [28]:
axis_index = {'lat': 0, 'lon': 2, 'lev': 1}

In [29]:
def rev(d : dict):
    '''Switches keys and values of dictionary d. Values of d must be hashable.
    '''
    
    return {v : k for k, v in d.items()}

In [30]:
rev(axis_index)

{0: 'lat', 2: 'lon', 1: 'lev'}

In [31]:
shape = (20, 10, 30)

In [48]:
next(iter(axis_index.items()))

('lat', 0)

In [36]:
tile_shape = []
raxd = rev(axis_index)
ks = list(raxd.keys()); ks.sort()
for idx in ks:
    if (raxd[idx] == 'lev'):
        tile_shape.append(-1)
    else:
        tile_shape.append(shape[idx])
tile_shape = tuple(tile_shape)

In [37]:
tile_shape

(20, -1, 30)

In [38]:
b = np.tile(a, tile_shape)

ValueError: negative dimensions are not allowed

In [35]:
b.shape

(20, 1, 900)

In [43]:
np.broadcast_to(a, (shape)).shape

(20, 10, 30)